In [5]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from ResNet import *
from run_model import *

import time
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, models, transforms
from collections import OrderedDict
from PIL import Image
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.4.0
Torchvision Version:  0.5.0


In [6]:
from dataloader import RetinopathyLoader

In [7]:
train_set = RetinopathyLoader(root='./data/',mode='train')
train_loader = data.DataLoader(dataset=train_set,batch_size=4)
test_set = RetinopathyLoader(root='./data/',mode='test')
test_loader = data.DataLoader(dataset=test_set,batch_size=4)

dataloaders_dict = {'train':train_loader,
                    'val' : test_loader}

> Found 28099 images...
> Found 7025 images...


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet18"

# Number of classes in the dataset
num_classes = 5

# Batch size for training (change depending on how much memory you have)
batch_size = 1

# Number of epochs to train for 
num_epochs = 10

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [10]:
def get_pretrained_model(model_name, num_classes,use_pretrained=True):
    model_dict = {
        'resnet18':  models.resnet18,
        'resnet34':  models.resnet34,
        'resnet50':  models.resnet50,
        'resnet101': models.resnet101,
        'resnet152': models.resnet152
    }
    
    model_ft = model_dict[model_name](pretrained=use_pretrained)          
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    
    return model_ft, input_size

## Run ResNet pretrained model which is from `torchvision`

In [7]:
# Initialize the pretrained model for this run
model_ft, input_size = get_pretrained_model(model_name, num_classes=5)


# Setup optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9)

# Send the model to GPU
model_ft = model_ft.to(device)

# Train and evaluate pretrained model
pretrained_loss_list, pretrained_acc_train_list, pretrained_acc_test_list = \
    run(model_ft, dataloaders_dict, criterion, num_epochs=num_epochs)

Epoch 1/10
----------
train Loss: 0.7780 Acc: 0.7416
val Loss: 0.6991 Acc: 0.7577
Epoch 2/10
----------
train Loss: 0.6484 Acc: 0.7797
val Loss: 0.6766 Acc: 0.7741
Epoch 3/10
----------
train Loss: 0.5958 Acc: 0.7985
val Loss: 0.6435 Acc: 0.7900
Epoch 4/10
----------
train Loss: 0.5552 Acc: 0.8114
val Loss: 0.6063 Acc: 0.7984
Epoch 5/10
----------
train Loss: 0.5166 Acc: 0.8232
val Loss: 0.5972 Acc: 0.8040
Epoch 6/10
----------
train Loss: 0.4698 Acc: 0.8385
val Loss: 0.6039 Acc: 0.8009
Epoch 7/10
----------
train Loss: 0.4201 Acc: 0.8529
val Loss: 0.6386 Acc: 0.7994
Epoch 8/10
----------
train Loss: 0.3673 Acc: 0.8696
val Loss: 0.6942 Acc: 0.7927
Epoch 9/10
----------
train Loss: 0.3028 Acc: 0.8925
val Loss: 0.7144 Acc: 0.7949
Epoch 10/10
----------
train Loss: 0.2363 Acc: 0.9175
val Loss: 0.8592 Acc: 0.7284


In [10]:
torch.save(model_ft,'./models/vision_pretrained_resnet18')
torch.save(pretrained_loss_list,'./result_list/pretrained_loss_list')
torch.save(pretrained_acc_train_list, './result_list/pretrained_acc_train_list')
torch.save(pretrained_acc_test_list, './result_list/pretrained_acc_test_list')

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:293: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:293: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:293: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site-packages/torch/serialization.py:293: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/opt/conda/lib/python3.6/site

## Run ResNet model (no pretrained) which is from `torchvision`

In [ ]:
# Initialize the non-pretrained version of the model used for this run
scratch_model,_ = get_pretrained_model(model_name, num_classes=5, use_pretrained=False)
scratch_model = scratch_model.to(device)
scratch_loss_list, scratch_acc_train_list, scratch_acc_test_list = \
    run(scratch_model, dataloaders_dict, criterion, num_epochs=num_epochs)

In [ ]:
torch.save(scratch_model,'./models/vision_not_pretrained_resnet18')

## Run ResNet implemented by myself

In [11]:
MyResNet = ResNet18(num_classes=5)
MyResNet = MyResNet.to(device)

In [23]:
############# ! CUDA ##############
my_loss_list, my_acc_train_list, my_acc_test_list = \
    run(MyResNet, dataloaders_dict, criterion, num_epochs=num_epochs)


Epoch 1/10
----------


KeyboardInterrupt: 

In [ ]:

torch.save(MyResNet,'./models/my_resnet18')50
torch.save(my_loss_list,'./result_list/my_loss_list_18')
torch.save(my_acc_train_list, './result_list/my_acc_train_list_18')
torch.save(my_acc_test_list, './result_list/my_acc_test_list_18')

## Comparison

In [ ]:
ohist  = [h.cpu().numpy() for h in pretrained_acc_test_list]
#shist  = [h.cpu().numpy() for h in scratch_acc_test_list]
myhist = [h.cpu().numpy() for h in my_acc_test_list]

plt.title("Validation Accuracy vs. Number of Training Epochs")
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
plt.plot(range(1,num_epochs+1),ohist,label="Pretrained")
#plt.plot(range(1,num_epochs+1),shist,label="Scratch")
plt.plot(range(1,num_epochs+1),myhist,label="My model")
plt.ylim((0,1.))
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.legend()
plt.show()

## ResNet 50

In [ ]:
# Initialize the pretrained model for this run
model_ft_50, input_size = get_pretrained_model('resnet50', num_classes=5)


# Setup optimizer
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9)

# Send the model to GPU
model_ft_50 = model_ft_50.to(device)

# Train and evaluate pretrained model
pretrained_loss_list_50, pretrained_acc_train_list_50, pretrained_acc_test_list_50 = \
    run(model_ft_50, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(model_ft_50,'./models/vision_pretrained_resnet50')
torch.save(pretrained_loss_list_50,'./result_list/pretrained_loss_list_50')
torch.save(pretrained_acc_train_list_50, './result_list/pretrained_acc_train_list_50')
torch.save(pretrained_acc_test_list_50, './result_list/pretrained_acc_test_list_50')

In [ ]:
MyResNet50 = ResNet50(num_classes=5)
MyResNet50 = MyResNet50.to(device)
my_loss_list_50, my_acc_train_list_50, my_acc_test_list_50 = \
    run(MyResNet50, dataloaders_dict, criterion, num_epochs=5)

In [ ]:
torch.save(MyResNet50,'./models/my_resnet50')
torch.save(my_loss_list_50,'./result_list/my_loss_list_50')
torch.save(my_acc_train_list_50, './result_list/my_acc_train_list_50')
torch.save(my_acc_test_list_50, './result_list/my_acc_test_list_50')

In [ ]:
MyResNet101 = ResNet101(num_classes=5)
MyResNet101 = MyResNet101.to(device)
my_loss_list_101, my_acc_train_list_101, my_acc_test_list_101 = \
    run(MyResNet101, dataloaders_dict, criterion, num_epochs=num_epochs)

In [ ]:
MyResNet152 = ResNet152(num_classes=5)
MyResNet152 = MyResNet152.to(device)
my_loss_list_152, my_acc_train_list_152, my_acc_test_list_152 = \
    run(MyResNet15, dataloaders_dict, criterion, num_epochs=num_epochs)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)